In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

In [2]:
# options for mgf
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""

options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)


rhf_energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=5, triplets = "NONE", tda=True)


# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

psi4_tdm = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]


print(rhf_energy)
print(rhf_energy + psi4_excitation_e[0])

-199.86395910419137
-199.69011028328705


In [3]:
print(psi4_tdm[0])

[-6.93218490e-16 -1.77990759e-15  2.33258251e+00]


In [4]:
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""

options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'davidson_roots' : 8,
    'davidson_threshold' : 1e-8,
    'full_diagonalization' : True,
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)

expected_e4 = -199.69011028328705

#energy from psi4numpy
expected_mu_04 = np.array([-6.93218490e-16, -1.77990759e-15,  2.33258251e+00])


test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)

actual_e0 = test_pf.CIeigs[0]
actual_e4 = test_pf.CIeigs[4]
print(actual_e0)
print(actual_e4)

actual_mu_04 = test_pf.compute_dipole_moment(0, 3)
print(actual_mu_04)
#assert np.isclose(actual_e4, expected_e4)
#assert np.allclose(actual_mu_04, expected_mu_04)


Start SCF iterations:

Canonical RHF One-electron energy = -281.6658725092592590
CQED-RHF One-electron energy      = -281.6658725092592590
Nuclear repulsion energy          = 2.8864211491090908
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -199.8639591041915651   dE = -1.99864E+02   dRMS = 1.51684E-12
SCF Iteration   2: Energy = -199.8639591041911103   dE =  4.54747E-13   dRMS = 2.67192E-13
Total time for SCF iterations: 0.002 seconds 

QED-RHF   energy: -199.86395910 hartree
Psi4  SCF energy: -199.86395910 hartree
 Completed QED-RHF in 0.7835681438446045 seconds
 Completed 1HSO Build in 0.00028228759765625 seconds
 Completed ERI Build in 0.49338388442993164 seconds 
 Completed 2D build in 0.02411484718322754 seconds
 Completed 1G build in 8.0108642578125e-05 seconds
 Completed the Dipole Matrix Build in 0.0007822513580322266 seconds
 Completed determinant list in 0.0068819522857666016 seconds 
 Completed constant offset matrix in 0.000159025192260

In [5]:
actual_mu_06 = test_pf.compute_dipole_moment(0, 4)
print(actual_mu_06)

[-6.58836036e-13  1.72314848e-13 -2.33258235e+00]


In [6]:
print(test_pf.rhf_dipole_moment)

[ 1.23668023e-15  3.26291298e-15 -1.50279429e+00]


In [11]:
actual_mu_g = test_pf.compute_dipole_moment(4, 4) + test_pf.mu_nuc
print(actual_mu_g)

[ 1.05705114e-12 -3.19182355e-13  2.07947966e+00]
